In [1]:
# Imports needed for this ipython file

from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
import pickle
from TwitterAPI import TwitterAPI

%matplotlib inline

In [2]:
def get_twitter(config_file):
    """ 
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [3]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)

In [4]:
def get_followers(screen_name):
    """ Return a dictionary of the users that follows this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A dictionary of dictionary, one per follower containing {index, user_info} pairs.
    Note: Many users follow more than 200 accounts; we will return those accounts too.
    """
    followers = {}
    cursor = -1
    key = 0
    rate_limit = 1
    while True:
        if rate_limit == 15:
            rate_limit = 1
            time.sleep(61 * 15)
            print >> sys.stderr, 'Avoided rate_limit error: \nsleeping for 15 minutes.'
        request = robust_request(twitter,'followers/list', 
                                 {'screen_name': screen_name, 'count':200, 'cursor': cursor})
        json_response = request.json()
        
        # Get 200 followers from json_response
        for follower in json_response['users']:
                followers[key] = follower
                key = key + 1
                
        # If user has < 200 followers or to retrieve last 200 followers
        if json_response["next_cursor"] == 0:
            break
            
        # Update cursor to get next 200 followers
        cursor = json_response["next_cursor"]
        rate_limit = rate_limit + 1
        print "Collected %d followers" % key
    return followers

In [61]:
# Getting iit alumni association followers and pickling them
iit_alumni_followers = get_followers('iitalumniassn')
iitalumniassn_fileObject = open("raw_data/iitalumniassn",'wb')
pickle.dump(iit_alumni_followers,iitalumniassn_fileObject)
iitalumniassn_fileObject.close()

In [9]:
fileObject = open("raw_data/iitalumniassn",'r')
iit_alumni_followers = pickle.load(fileObject)
fileObject.close()
print "Found %d followers for iitalumniassn" %len(iit_alumni_followers.keys())

Found 1861 followers for iitalumniassn


In [6]:
# Getting iit career services followers and pickling them
iit_career_followers = get_followers('IITCareers')
iitcareerservices_fileObject = open("raw_data/iitcareerservices",'wb')
pickle.dump(iit_career_followers,iitcareerservices_fileObject)
iitcareerservices_fileObject.close()

In [5]:
fileObject = open("raw_data/iitcareerservices",'r')
iit_career_followers = pickle.load(fileObject)
fileObject.close()
print "Found %d followers for IITCareers" %len(iit_career_followers.keys())

Found 642 followers for IITCareers


In [ ]:
# Getting iit main university followers and pickling them
iit_univ_followers = get_followers('illinoistech')
iituniv_fileObject = open("raw_data/iituniv",'wb')
pickle.dump(iit_univ_followers,iituniv_fileObject)
iituniv_fileObject.close()

In [10]:
fileObject = open("raw_data/iituniv",'r')
iit_univ_followers = pickle.load(fileObject)
fileObject.close()
print "Found %d followers for IIT University" %len(iit_univ_followers.keys())

Found 5426 followers for IIT University


In [ ]:
# Getting uiuc main university followers and pickling them
uiuc_univ_followers = get_followers('Illinois_Alma')
uiuc_univ_fileObject = open("raw_data/uiucuniv",'wb')
pickle.dump(uiuc_univ_followers,uiuc_univ_fileObject)
uiuc_univ_fileObject.close()

In [16]:
fileObject = open("raw_data/uiucuniv",'r')
uiuc_univ_followers = pickle.load(fileObject)
fileObject.close()
print "Found %d followers for UIUC University" %len(uiuc_univ_followers.keys())

Found 40017 followers for UIUC University


In [10]:
for key in range(len(iit_alumni_followers[key].keys())):
    user = iit_alumni_followers[key]
    screen_name = user['screen_name']

{0: {u'blocked_by': False,
  u'blocking': False,
  u'contributors_enabled': False,
  u'created_at': u'Sun Sep 14 14:39:56 +0000 2014',
  u'default_profile': True,
  u'default_profile_image': True,
  u'description': u'',
  u'entities': {u'description': {u'urls': []}},
  u'favourites_count': 12,
  u'follow_request_sent': False,
  u'followers_count': 1,
  u'following': False,
  u'friends_count': 24,
  u'geo_enabled': False,
  u'has_extended_profile': False,
  u'id': 2809550070L,
  u'id_str': u'2809550070',
  u'is_translation_enabled': False,
  u'is_translator': False,
  u'lang': u'zh-cn',
  u'listed_count': 0,
  u'location': u'',
  u'muting': False,
  u'name': u'Zhe Wang',
  u'notifications': False,
  u'profile_background_color': u'C0DEED',
  u'profile_background_image_url': u'http://abs.twimg.com/images/themes/theme1/bg.png',
  u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png',
  u'profile_background_tile': False,
  u'profile_image_url': u'http:/

In [ ]:
def get_student_screen_names(*args):
    student_screen_names = set()
    for univ_followers in args:
        for user in univ_followers.values():
            

In [6]:
def get_followers_and_pickle_object(account_name):
    obj = get_followers(account_name)
    fileObject = open("raw_data/" + account_name,'wb')
    pickle.dump(obj,fileObject)
    fileObject.close()

In [7]:
def get_followers_from_pickle(account_name):
    fileObject = open("raw_data/" + account_name,'r')
    followers = pickle.load(fileObject)
    fileObject.close()
    print "Found %d followers" %len(followers.keys())
    return followers

In [13]:
iitalumniassn_followers = get_followers_from_pickle("iituniv")
new_iitalumniassn_followers = {}
for value in iitalumniassn_followers.values():
    new_iitalumniassn_followers[value["screen_name"]] = value

Found 643 followers


In [14]:
fileObject = open("raw_data/" + "IITCareers",'wb')
pickle.dump(new_iitalumniassn_followers,fileObject)
fileObject.close()

In [18]:
fileObject = open("raw_data/iituniv",'r')
iit_univ_followers = pickle.load(fileObject)
fileObject.close()